# Consistency and tension
Aim: Determine the level of consistency between two datasets

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
from chainconsumer import ChainConsumer
# Sampler
import pymultinest
# If you haven't installed MultiNest: use conda install -c conda-forge pymultinest 

In this tutorial we will work with some mock data resembling a cosmic shear measurement. In cosmic shear we measure the galaxy shapes in several tomographic redshift bins. From these measurements we calculate two-point statistics, from which we then constrain cosmological parameters. 

Here, we will work with one single redshift bins and pretend that we have two cosmic shear measurements in this bin. Our goal is to asses whether or not the two observations are consistent with each other and if there exists a model that provides a good fit to both datasets at the same time. For simplicity, we will not model the full cosmic shear power spectrum, but instead adopt a very simple model.

There are two sets of data in this directory (data_1.txt and data_2.txt). Each file contains three columns: 8 logarithmic $\ell$-values and two corresponding measurements of cosmic shear band power spectra $C_E$ for dataset A and B. As we will see later, the measurements in one file are consistent with each other while the other ones show some tension. For simplicity, we assume all measurements share the covariance matrix in covariance.txt.

In [ ]:
# Read the data in data_1.txt and the covariance matrix in covariance_1.txt


Now, plot the two data vectors using matplotlib (including errorbars)

In [ ]:
# Plot the two datavectors (note: plot ell vs data/ell for better visualisation)


Our goal is to fit a theory model to the two observed datasets and quantify the level of consistency between the two experiments. We will use MultiNest to sample the posterior and to infer the evidence.

In [ ]:
# Define a linear model simple model 
def model(ell, a, b):
    return((b + ell*a)*1e-5)

MultiNest generates samples in a unit hyper-cube in which all the parameter are uniformly distributed in the interval [0, 1].
Therefore, we need to define a function that transforms the interval [0,1] onto our desired prior. Additionally, we need to define the likelihood function.

To get started, define a uniform prior in the interval [-2.5,9] for the amplitude and [0.022,0.055] for the tilt.

The MultiNest prior function and the likelihood function require three inputs: the MultiNest hypercube, the number of dimensions, and the number of parameters.
The number of parameters and the number of dimension are the same, unless we want to generate a set of derived parameters. 

The prior function should return the transformed hypercube and the likelihood function should return the log-likelihood.

In [ ]:
# Define the prior function
# The prior function should use the following inputs:
# cube: A N-dimension array (where N is the number of model parameters). The entries of this array are the parameter values
#       generated by MultiNest during sampling. However, MultiNest generates its parameter values in the interval [0,1],
#       which is why we need to define a transformation onto our prior interval. 
# ndim: The number of model parameters
# nparams: The total number of parameters (which can be larger than the number of model parameters if we calculate
#          some derived parameters). In this tutorial this parameter will be equal to ndim.
# The output of the function should be the transformed N-dimensional array
def prior(cube, ndim):
    # define a linear transformation from [0,1] to the prior interval

    return(transformed_cube)
    
# Define the likelihood function (hint: use a Gaussian chi^2)
# The likelihood function should use the following inputs:
# cube: A N-dimensional array, which contains the values of our model parameters for a given point.
#       (Internally, MultiNest first generates parameters in the interval [0,1], which are passed to the prior 
#       function (defined above). The output of the prior function (i.e. the transformed parameter values) is then
#       passed to the likelihood function.
# ndim: same as above
# nparams: same as above

def log_lkl(cube, ndim, nparams):
    # Define a Gaussian log-likelihood with the data that you loaded from the text file

    # return the log-likelihood
    return(loglkl)


In [ ]:
# MultiNest parameters
eff = 0.1
tol = 0.001
nLive = 1000
nDims = 2

In [ ]:
# Sample the likelihood using multinest
# Hint: use pymultinest.run (you can run 'help(pymultinest.run)' to check how to run the sampler.
# MultiNest automatically saves some output files to disk. You need to set outputfiles_basename for each chain.

# IMPORTANT NOTE: If the output folder doesn't exist, MultiNest WILL CRASH! 
#                 Make sure to create output folder before running MultiNest!

# Additionally, MultiNest requires you to set the following parameters: n_live_points, sampling_efficiency, evidence_tolerance.
# Use the values provided in the cell above




Now plot the chains using chainconsumer

In [ ]:
# Read the chains
# The chains contain 2+nparams columns: weight, -2*loglikehood, parameter values
# Hint: pymultinest.Analyzer allows you to access the MultiNest output. Alternatively, you can directly load the chain from outputfiles_basename.txt 



In [ ]:
# To visualize the prior, run another chain with constant likelihood to generate samples from the prior
# Let's just set up a dummy likelihood function that just return a constant value.

# Run MultiNest with this sampler to generate a chain that resembles our prior.


In [ ]:
# Plot the chains, including the prior chain
# Hint: Use chainconsumer (You can look up the syntax in the session 1 notebook)
# Note: Each point in the MultiNest chain carries a corresponding weight. These need to be passed to chainconsumer as additional input



It is useful to plot the translated posterior distribution (TPD) to visualize the range of theory vectors in the chain. This requires us to translate each sample in the chain back into the data domain.

In [ ]:
# For each chain, evaluate the model at every point 
# The result should be a (N_points, N_data)-dimensional array, where N_points is the number of points in the chain and N_data is the number of data points



In [ ]:
# Plot the data vector together with the mean and the standard deviation of the of the TPD. 
# Hint: Remember to take the weights into account when calculating the mean and standard deviation.
# Use plt.fill_between to plot the 1-sigma interval of the TPD

# You can use this package to calculate a weighted mean and standard deviation
from statsmodels.stats.weightstats import DescrStatsW

# Alternatively, you can use weights in np.average. However, np.std doesn't support weighted samples. You could try to calculate the standard deviation by yourself.


Now, we want to test if the two datasets are consistent with each other. To do so, we test the following hypotheses:
1) There exist two sets of parameters that each describe one dataset independently
2) There exists one single set of parameters that describes both datasets A and B at the same time

So far, our chains have tested hypothesis 2). Next, we will run chains testing hypothesis 1).

In [ ]:
# Define the likelihood function for the combination of datasets A and B (hint: log-likelihoods are additive)
def log_lkl(cube, ndim, nparams):

    return(loglkl)

In [ ]:
# Sample the combined likelihood with a single set of parameters


In [ ]:
# Read the chains


In [ ]:
# Plot the chains


Now that we have run all chains we can make a model comparison to infer which of the two hypotheses is preferred by the data.
We calculate the Bayes Ratio:

$\log R = \log Z_{AB} - \log Z_{A} - \log Z_{B}$

In [ ]:
# Calculate Bayes ratio 
# Hint: Use the get_stats() function of the pymultinest.Analyzer class to get the evidence. 
# Alternatively, you can take a look at the output files produced by MultiNest

print('logR = %.2f'%logR)

Based on the Bayes ratios, in which case are the two datasets consistent with each other?

Now we want to investigate the impact of the prior volume on the Bayes ratio. 

In [ ]:
# Define a few sets of priors. Take a look at the posterior plot that you produced earlier and define some new priors that are broader/narrower.
def prior_wide(cube, ndim, nparams):
    
    return cube
# Define as many priors as you want!



In [ ]:
# Sample the likelihoods with different priors
# Individual likelihoods


In [ ]:
# Plot the new chains



In [ ]:
# Now calculate the Bayes ratio for each of your priors. How do the values change for different prior ranges?
